In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect('task1.sqlite')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE actors (
                id INTEGER PRIMARY KEY,
                name TEXT )
''')
cursor.execute('''CREATE TABLE movies (
                id INTEGER PRIMARY KEY,
                title TEXT )
''')
cursor.execute('''CREATE TABLE actor_movie (
               actor_id INTEGER,
               movie_id INTEGER,
               FOREIGN KEY (actor_id) REFERENCES actors(id),
               FOREIGN KEY (movie_id) REFERENCES movies(id) )
''')
connection.commit()
connection.close()

In [12]:
connectionNetflix = sqlite3.connect('netflix.sqlite')
cursorNetflix = connectionNetflix.cursor()
cursorNetflix.execute('select title, "cast" from netflix_titles')
data = cursorNetflix.fetchall()
connectionNetflix.close()

import re
updateData = []
uniqueActors = set()
for i in data:
    film, actors = i
    cast = re.split(r' ,|, |,', actors)
    uniqueActors = uniqueActors.union(set(cast))
    updateData.append((film, cast))

In [13]:
connectionActors = sqlite3.connect('task1.sqlite')
cursorActors = connectionActors.cursor()
for actor in uniqueActors:
    cursorActors.execute("INSERT INTO actors VALUES (NULL, ?)", (actor,))
connectionActors.commit()
connectionActors.close()

In [15]:
connectionFilms = sqlite3.connect('task1.sqlite')
cursorMovies = connectionFilms.cursor()
for i in updateData:
    film, actors = i
    cursorMovies.execute("INSERT INTO movies VALUES (NULL, ?)", (film,))
connectionFilms.commit()
connectionFilms.close()

In [16]:
connectionActorFilm = sqlite3.connect('task1.sqlite')
cursorActors = connectionActorFilm.cursor()
cursorFilms = connectionActorFilm.cursor()
cursorActorFilm = connectionActorFilm.cursor()
for i in updateData:
    film, actors = i
    cursorActorFilm.execute('SELECT id FROM movies WHERE title = ?', (film,))
    uniqueFilm = cursorActorFilm.fetchone()
    for actor in actors:
        cursorActors.execute('SELECT id FROM actors WHERE name = ?', (actor,))
        uniqueActor = cursorActors.fetchone()
        cursorActorFilm.execute("INSERT INTO actor_movie (actor_id, movie_id) VALUES (?, ?)", (uniqueActor[0], uniqueFilm[0]))
connectionActorFilm.commit()
connectionActorFilm.close()

In [18]:
connectionFindActors = sqlite3.connect('task1.sqlite')
cursorFindActors = connectionFindActors.cursor()
cursorFindActors.execute('''
    SELECT a1.name AS actor1_name, a2.name AS actor2_name, COUNT(*) AS movies_together
    FROM actor_movie am1
    JOIN actor_movie am2 ON am1.movie_id = am2.movie_id AND am1.actor_id < am2.actor_id
    JOIN actors a1 ON am1.actor_id = a1.id
    JOIN actors a2 ON am2.actor_id = a2.id
    GROUP BY actor1_name, actor2_name
    ORDER BY movies_together DESC
    LIMIT 1
''')
actors = cursorFindActors.fetchone()
print(f"Наиболее часто работающая друг с другом пара актеров: {actors[0]} и {actors[1]}")
connectionFindActors.close()

Наиболее часто работающая друг с другом пара актеров: Robb Wells и John Paul Tremblay
